In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

In [2]:
horse_data = pd.read_csv('./datacarreras.csv', delimiter=',')
horse_data.head()

,fecha,numero_carrera,distancia_pista,grupo,nombre_caballo,tiempo,ult_div,penult_div,posicion,cuerpos_ult,cuerpos_penult,distancia_c,lote,jinete_act,jinete_gana,jinete_rep_mon,trainer_act,trainer_gana,resultado
0,08-11-2020,1,1800,2,ironhide,113.0,2100,2300,2,4.25,8.50,2,2,122,16,3,49,5,6
1,08-11-2020,1,1800,2,el ferretero,114.2,10000,10000,7,11.25,13.00,1,3,69,1,1,50,0,3
2,08-11-2020,1,1800,2,matheew star,115.0,350,350,5,6.50,4.25,2,3,43,5,3,93,8,4
3,08-11-2020,1,1800,2,power indy,113.3,450,600,3,6.50,5.50,3,3,99,14,1,146,19,1
4,08-11-2020,1,1800,2,gran giacomo,115.2,540,540,1,0.00,0.50,2,1,92,23,1,31,6,8


In [3]:
# Wins/Starts
horse_data['effect_jinete'] = round(horse_data["jinete_gana"] / horse_data["jinete_act"] * 100, 2)
horse_data['effect_trainer'] = round(horse_data["trainer_gana"] / horse_data["trainer_act"] * 100, 2)

# Filtering and replacing values

horse_data = horse_data.replace(np.inf, 30)
horse_data = horse_data[(horse_data.resultado != 0)]
horse_data["jinete_rep_mon"].replace(1, 0)
horse_data["jinete_rep_mon"].replace(3, 1)

# Amelioration of dividends with regards to last race
horse_data["mej_div"] = horse_data["penult_div"] - horse_data["ult_div"]

horse_data["distancia_pista"].value_counts()

1200    669
1300    425
1100    367
1400    353
1500     66
1800     58
1600     57
2400     23
1900     12
2000      8
Name: distancia_pista, dtype: int64

In [4]:
horse_data = horse_data[['fecha', 'numero_carrera', 'distancia_pista', 'tiempo', 'penult_div','ult_div', 'mej_div','posicion','cuerpos_ult', 'distancia_c', 'lote', 'jinete_gana', 'effect_jinete', 'jinete_rep_mon', 'trainer_gana','effect_trainer', 'resultado']]
horse_data["ult_div"].replace(0, horse_data["ult_div"].mean(axis=0), inplace=True)
horse_data["mej_div"].replace(0, horse_data["mej_div"].mean(axis=0), inplace=True)
horse_data.head()

,fecha,numero_carrera,distancia_pista,tiempo,penult_div,ult_div,mej_div,posicion,cuerpos_ult,distancia_c,lote,jinete_gana,effect_jinete,jinete_rep_mon,trainer_gana,effect_trainer,resultado
0,08-11-2020,1,1800,113.0,2300,2100.0,200.000000,2,4.25,2,2,16,13.11,3,5,10.20,6
1,08-11-2020,1,1800,114.2,10000,10000.0,-149.041708,7,11.25,1,3,1,1.45,1,0,0.00,3
2,08-11-2020,1,1800,115.0,350,350.0,-149.041708,5,6.50,2,3,5,11.63,3,8,8.60,4
3,08-11-2020,1,1800,113.3,600,450.0,150.000000,3,6.50,3,3,14,14.14,1,19,13.01,1
4,08-11-2020,1,1800,115.2,540,540.0,-149.041708,1,0.00,2,1,23,25.00,1,6,19.35,8


In [5]:
train_data = horse_data.iloc[:1844]
test_data = horse_data.iloc[1844:]

In [6]:
train_data.tail()

,fecha,numero_carrera,distancia_pista,tiempo,penult_div,ult_div,mej_div,posicion,cuerpos_ult,distancia_c,lote,jinete_gana,effect_jinete,jinete_rep_mon,trainer_gana,effect_trainer,resultado
1941,04-04-2021,11,1200,75.0,5700,600.0,5100.000000,3,4.00,3,3,14,10.00,3,7,7.78,4
1942,04-04-2021,11,1200,77.1,10000,10000.0,-149.041708,6,10.50,1,3,0,0.00,1,2,3.08,11
1943,04-04-2021,11,1200,76.2,10000,600.0,9400.000000,7,14.00,1,3,4,15.38,1,28,14.51,5
1944,04-04-2021,11,1200,75.2,600,800.0,-200.000000,6,11.25,3,3,0,0.00,1,1,12.50,2
1945,04-04-2021,11,1200,75.4,5200,350.0,4850.000000,6,9.50,3,3,4,14.81,1,3,4.76,6


In [7]:
test_data.head()

,fecha,numero_carrera,distancia_pista,tiempo,penult_div,ult_div,mej_div,posicion,cuerpos_ult,distancia_c,lote,jinete_gana,effect_jinete,jinete_rep_mon,trainer_gana,effect_trainer,resultado
1946,11-04-2021,1,1400,88.4,1500,1500.0,-149.041708,4,10.75,3,3,9,13.24,3,10,14.49,1
1947,11-04-2021,1,1400,89.3,7200,4900.0,2300.000000,3,2.00,3,3,19,21.84,1,10,14.49,4
1948,11-04-2021,1,1400,90.2,10000,6900.0,3100.000000,4,15.75,1,3,2,10.00,3,2,6.06,2
1949,11-04-2021,1,1400,89.2,9600,10000.0,-400.000000,7,8.50,1,3,24,26.67,1,6,22.22,3
1950,11-04-2021,1,1400,90.4,10000,10000.0,-149.041708,8,27.25,1,3,1,4.35,1,4,5.00,6


In [8]:
train_data['resultado'] = train_data['resultado'] - 1
test_data['resultado'] = test_data['resultado'] - 1

train_data_y = train_data['resultado'].copy()
test_data_y = test_data['resultado'].copy()

train_data = train_data.drop(labels='resultado', axis=1)
test_data = test_data.drop(labels='resultado', axis=1)

train_data_y.value_counts()

C:\Users\Nevio Gomez\AppData\Local\Temp\ipykernel_11248\3304985669.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['resultado'] = train_data['resultado'] - 1
C:\Users\Nevio Gomez\AppData\Local\Temp\ipykernel_11248\3304985669.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['resultado'] = test_data['resultado'] - 1


2     187
0     184
1     182
3     177
4     177
5     171
6     165
7     151
8     134
9     115
10     82
11     66
12     38
13     15
Name: resultado, dtype: int64

In [9]:
train_data_y = tf.keras.utils.to_categorical(train_data_y)
test_data_y = tf.keras.utils.to_categorical(test_data_y)
train_data_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
train_data["fecha"] = (train_data["fecha"].apply(lambda x: x.replace("-", "").strip()).astype(np.int64))
unique_fechas_train = train_data["fecha"].unique()
test_data["fecha"] = (test_data["fecha"].apply(lambda x: x.replace("-", "").strip()).astype(np.int64))
unique_fechas_test = test_data["fecha"].unique()
unique_fechas_train

array([ 8112020, 15112020, 12122020, 13122020, 19122020, 20122020,
       10012021, 17012021, 30012021,  7022021, 14022021, 21022021,
       28022021,  7032021, 14032021, 21032021, 28032021,  4042021],
      dtype=int64)

In [11]:
train_data = train_data.groupby(["fecha", "numero_carrera"])
test_data = test_data.groupby(["fecha", "numero_carrera"])

In [12]:
train_data.get_group((8112020, 1))

,fecha,numero_carrera,distancia_pista,tiempo,penult_div,ult_div,mej_div,posicion,cuerpos_ult,distancia_c,lote,jinete_gana,effect_jinete,jinete_rep_mon,trainer_gana,effect_trainer
0,8112020,1,1800,113.0,2300,2100.0,200.000000,2,4.25,2,2,16,13.11,3,5,10.20
1,8112020,1,1800,114.2,10000,10000.0,-149.041708,7,11.25,1,3,1,1.45,1,0,0.00
2,8112020,1,1800,115.0,350,350.0,-149.041708,5,6.50,2,3,5,11.63,3,8,8.60
3,8112020,1,1800,113.3,600,450.0,150.000000,3,6.50,3,3,14,14.14,1,19,13.01
4,8112020,1,1800,115.2,540,540.0,-149.041708,1,0.00,2,1,23,25.00,1,6,19.35
5,8112020,1,1800,114.0,220,200.0,20.000000,4,9.00,3,3,0,0.00,1,19,13.01
6,8112020,1,1800,119.2,10000,6300.0,3700.000000,11,11.00,3,2,13,11.02,3,6,8.70
7,8112020,1,1800,117.5,10000,10000.0,-149.041708,11,15.25,1,3,1,11.11,1,0,0.00
8,8112020,1,1800,114.2,10000,10000.0,-149.041708,5,6.25,0,3,6,6.82,3,2,6.45


In [13]:
data_carreras_train = []
data_carreras_test = []

def getRaces(dataframe, num_races, races_list):
    for i in np.array(num_races):
        for r in range(1, 13):
            try:
                df = dataframe.get_group((i, r))
                df = df.iloc[:, -14:]
                df = tf.convert_to_tensor(df, dtype=tf.float64)
                races_list.append(df)
            except KeyError:
                print(f"No data found for ID: {r}, Race num: {i}")

In [14]:
getRaces(train_data, unique_fechas_train[:2], data_carreras_train)
data_carreras_train[:2]

No data found for ID: 2, Race num: 8112020
No data found for ID: 11, Race num: 15112020
No data found for ID: 12, Race num: 15112020


[<tf.Tensor: shape=(9, 14), dtype=float64, numpy=
 array([[ 1.80000000e+03,  1.13000000e+02,  2.30000000e+03,
          2.10000000e+03,  2.00000000e+02,  2.00000000e+00,
          4.25000000e+00,  2.00000000e+00,  2.00000000e+00,
          1.60000000e+01,  1.31100000e+01,  3.00000000e+00,
          5.00000000e+00,  1.02000000e+01],
        [ 1.80000000e+03,  1.14200000e+02,  1.00000000e+04,
          1.00000000e+04, -1.49041708e+02,  7.00000000e+00,
          1.12500000e+01,  1.00000000e+00,  3.00000000e+00,
          1.00000000e+00,  1.45000000e+00,  1.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 1.80000000e+03,  1.15000000e+02,  3.50000000e+02,
          3.50000000e+02, -1.49041708e+02,  5.00000000e+00,
          6.50000000e+00,  2.00000000e+00,  3.00000000e+00,
          5.00000000e+00,  1.16300000e+01,  3.00000000e+00,
          8.00000000e+00,  8.60000000e+00],
        [ 1.80000000e+03,  1.13300000e+02,  6.00000000e+02,
          4.50000000e+02,  1.50000000e

In [15]:
data_carreras_train = np.vstack(data_carreras_train)
data_carreras_train

array([[1.800e+03, 1.130e+02, 2.300e+03, ..., 3.000e+00, 5.000e+00,
        1.020e+01],
       [1.800e+03, 1.142e+02, 1.000e+04, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [1.800e+03, 1.150e+02, 3.500e+02, ..., 3.000e+00, 8.000e+00,
        8.600e+00],
       ...,
       [1.100e+03, 6.710e+01, 1.000e+04, ..., 3.000e+00, 1.000e+01,
        1.754e+01],
       [1.100e+03, 6.830e+01, 1.000e+04, ..., 1.000e+00, 2.000e+01,
        1.408e+01],
       [1.100e+03, 6.800e+01, 1.300e+03, ..., 3.000e+00, 2.000e+00,
        6.670e+00]])

In [16]:
data_carreras_train.shape

(227, 14)